In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from skimage.io import imread

/Users/mingyuzhu/opt/anaconda3/envs/comp_graph_theory/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#create triangle of a given side length that is positioned upright
def create_triangle(min_len,max_len):
    side_length = np.random.random() * (max_len - min_len) + min_len
    base_coor = (0, 0)
    c = np.tan(np.pi / 6) * (side_length / 2)
    A = (base_coor[0], base_coor[1] + np.sqrt(3) / 2 * side_length - c)
    B = (base_coor[0] -side_length / 2, base_coor[1]  -c)
    C = (base_coor[0] + side_length / 2, base_coor[1]  -c)
    return np.array([A, B, C])

# Help determine if the vertex of the new triangle already exists in the graph,
# if exists, return the position symmetric in respect to the line connected by
# node1 and node2
def triangle_helper(node1,node2,vert_locs):
    node1 = np.array(node1)
    node2 = np.array(node2)
    center = np.array([(node1[0] + node2[0]) / 2, (node1[1] + node2[1])/2])

    vec = node1 - node2
    len_vec = (vec[0]**2 + vec[1]**2)**(0.5)
    dir = np.dot(np.array([[0,-1],[1,0]]), (vec / len_vec))


    pt1 = center + dir * np.sqrt(3) / 2 * len_vec
    pt2 = center + dir * np.sqrt(3) / 2 * len_vec * (-1)

    min_dis_1 = 10000
    closest_neighbor_1 = None
    min_dis_2 = 10000
    closest_neighbor_2 = None

    #check if pt1 is already in vert_locs
    for i in range(len(vert_locs)):
        vert = vert_locs[i]
        dis = np.sqrt((vert[0] - pt1[0])**2 + (vert[1] - pt1[1])**2)
        if (dis < min_dis_1):
            min_dis_1 = dis
            closest_neighbor_1 = i

    for i in range(len(vert_locs)):
        vert = vert_locs[i]
        dis = np.sqrt((vert[0] - pt2[0])**2 + (vert[1] - pt2[1])**2)
        if (dis < min_dis_2):
            min_dis_2 = dis
            closest_neighbor_2 = i

    #closure
    if (min_dis_1 < len_vec / 100 and min_dis_2 < len_vec/100):
        return True, [pt1,pt2,closest_neighbor_1,closest_neighbor_2]

    else:
        if (min_dis_1 <= len_vec / 100):
            return False, [pt2]
        else:
            return False, [pt1]

# Take in two nodes, return the center of the line that connects these two nodes
def edge_sampler(node1,node2):
    node1 = np.array(node1)
    node2 = np.array(node2)
    center = np.array([(node1[0] + node2[0]) / 2, (node1[1] + node2[1])/2])
    return int(center[0]),int(center[1])

In [3]:
# This function takes in all the information of a triangular mesh, then randomly adds only one more triangle to the existing mesh
def create_multi_triangle_rec(vertice_locations, edges, outer_edges, triangles):
    rand_idx = np.random.randint(0,len(outer_edges))
    orig_ver1 = list(outer_edges[rand_idx])[0]
    orig_ver2 = list(outer_edges[rand_idx])[1]
    closure, new_vertexs = triangle_helper(vertice_locations[orig_ver1],vertice_locations[orig_ver2], vertice_locations)

    operation = 0
    if closure == False:
        #operation 0
        new_vertex = new_vertexs[0]

        vertice_locations.append((new_vertex[0],new_vertex[1]))
        new_edge1 = set([orig_ver1,len(vertice_locations)-1])
        new_edge2 = set([orig_ver2,len(vertice_locations)-1])

        edges.append(new_edge1)
        edges.append(new_edge2)

        triangles.append(set([orig_ver1, orig_ver2, len(vertice_locations)-1]))

        outer_edges.remove(set([orig_ver1,orig_ver2]))
        outer_edges.append(new_edge1)
        outer_edges.append(new_edge2)


    else:
        clo_nei1 = new_vertexs[2]
        clo_nei2 = new_vertexs[3]

        #don't try to understand this
        c1o1 = set([clo_nei1, orig_ver1]) not in edges
        c1o2 = set([clo_nei1, orig_ver2]) not in edges
        c2o1 = set([clo_nei2, orig_ver1]) not in edges
        c2o2 = set([clo_nei2, orig_ver2]) not in edges


        #operation 1
        if ((not c1o1) and (not c1o2) ) and ((not c2o1) and (not c2o2)):
            if set([orig_ver1,orig_ver2,clo_nei1]) in triangles:
                triangles.append(set([clo_nei2,orig_ver1,orig_ver2]))

                outer_edges.remove(set([orig_ver1,orig_ver2]))
                outer_edges.remove(set([clo_nei2,orig_ver1]))
                outer_edges.remove(set([clo_nei2,orig_ver2]))


            else:
                triangles.append(set([clo_nei1,orig_ver1,orig_ver2]))

                outer_edges.remove(set([orig_ver1,orig_ver2]))
                outer_edges.remove(set([clo_nei1,orig_ver1]))
                outer_edges.remove(set([clo_nei1,orig_ver2]))

            operation = 1
        #operation 2
        elif (c1o1 and c1o2) or (c2o1 and c2o2):
            if (c1o1 and c1o2):
                new_edge1 = set([clo_nei1,orig_ver1])
                new_edge2 = set([clo_nei1,orig_ver2])

                edges.append(new_edge1)
                edges.append(new_edge2)

                triangles.append(set([clo_nei1,orig_ver1,orig_ver2]))

                outer_edges.remove(outer_edges[rand_idx])

                outer_edges.append(new_edge1)
                outer_edges.append(new_edge2)

            else:
                new_edge1 = set([clo_nei2,orig_ver1])
                new_edge2 = set([clo_nei2,orig_ver2])
                edges.append(new_edge1)
                edges.append(new_edge2)

                triangles.append(set([clo_nei2,orig_ver1,orig_ver2]))

                outer_edges.remove(outer_edges[rand_idx])

                outer_edges.append(new_edge1)
                outer_edges.append(new_edge2)

            operation = 2


        #operation 3
        else:
            if set([orig_ver1,clo_nei1]) not in edges:
                new_edge = set([orig_ver1,clo_nei1])

                edges.append(new_edge)

                triangles.append(set([orig_ver1,orig_ver2,clo_nei1]))

                outer_edges.remove(outer_edges[rand_idx])

                outer_edges.remove(set([clo_nei1,orig_ver2]))

                outer_edges.append(new_edge)


            elif set([orig_ver1,clo_nei2]) not in edges:
                new_edge = set([orig_ver1,clo_nei2])

                edges.append(new_edge)

                triangles.append(set([orig_ver1,orig_ver2,clo_nei2]))

                outer_edges.remove(outer_edges[rand_idx])
                outer_edges.remove(set([clo_nei2,orig_ver2]))

                outer_edges.append(new_edge)

            elif set([orig_ver2,clo_nei1]) not in edges:
                new_edge = set([orig_ver2,clo_nei1])
                edges.append(new_edge)

                triangles.append(set([orig_ver1,orig_ver2,clo_nei1]))

                outer_edges.remove(outer_edges[rand_idx])
                outer_edges.remove(set([clo_nei1,orig_ver1]))


                outer_edges.append(new_edge)
            else:
                new_edge = set([orig_ver2,clo_nei2])
                edges.append(new_edge)
                triangles.append(set([orig_ver1,orig_ver2,clo_nei2]))

                outer_edges.remove(outer_edges[rand_idx])
                outer_edges.remove(set([clo_nei2,orig_ver1]))


                outer_edges.append(new_edge)

            operation = 3

    return vertice_locations, edges, outer_edges, triangles, operation

In [138]:
# Checks if the coordinate (x,y) is within the highlighted part of the image
def point_valid(x,y,img):
    if (x >= 0 and x < img.shape[0] and y >= 0 and y < img.shape[1]):
        return img[int(x), int(y)] == 1
    else:
        return False

In [139]:
# This function is similar to create_multi_triangle_rec, but it generates a mesh of an image.
# It takes in two additional arguments -- boundary_edges and img, and adds only one triangle every time.
def create_multi_triangle_img_rec(vertice_locations, boundary_edges, edges, outer_edges, triangles, img):
    case_1_on = True
    case_2_on = True
    case_3_on = True

    num_remove_errors = 0

    rand_idx = np.random.randint(0,len(outer_edges))
    orig_ver1 = list(outer_edges[rand_idx])[0]
    orig_ver2 = list(outer_edges[rand_idx])[1]
    closure, new_vertexs = triangle_helper(vertice_locations[orig_ver1],vertice_locations[orig_ver2], vertice_locations)


    if closure == False:
        new_vertex = new_vertexs[0]

        if point_valid(new_vertex[0],new_vertex[1],img):

            vertice_locations.append((new_vertex[0],new_vertex[1]))
            new_edge1 = {orig_ver1, len(vertice_locations) - 1}
            new_edge2 = {orig_ver2, len(vertice_locations) - 1}

            edges.append(new_edge1)
            edges.append(new_edge2)

            triangles.append({orig_ver1, orig_ver2, len(vertice_locations) - 1})

            outer_edges.remove({orig_ver1, orig_ver2})
            outer_edges.append(new_edge1)
            outer_edges.append(new_edge2)


        else:
            outer_edges.remove({orig_ver1, orig_ver2})
            boundary_edges.append({orig_ver1, orig_ver2})


    else:
        clo_nei1 = new_vertexs[2]
        clo_nei2 = new_vertexs[3]

        #don't try to understand this
        c1o1 = {clo_nei1, orig_ver1} not in edges
        c1o2 = {clo_nei1, orig_ver2} not in edges
        c2o1 = {clo_nei2, orig_ver1} not in edges
        c2o2 = {clo_nei2, orig_ver2} not in edges

        #rnm case 1
        if ((not c1o1) and (not c1o2) ) and ((not c2o1) and (not c2o2)):
            if (case_1_on):
                if {orig_ver1, orig_ver2, clo_nei1} in triangles:
                    triangles.append({clo_nei2, orig_ver1, orig_ver2})

                    try:
                        outer_edges.remove({orig_ver1, orig_ver2})
                    except:
                        num_remove_errors += 1

                    try:
                        outer_edges.remove({clo_nei2, orig_ver1})
                    except:
                        num_remove_errors += 1

                    try:
                        outer_edges.remove({clo_nei2, orig_ver2})
                    except:
                        num_remove_errors += 1

                else:
                    triangles.append({clo_nei1, orig_ver1, orig_ver2})

                    try:
                        outer_edges.remove({orig_ver1, orig_ver2})
                    except:
                        num_remove_errors += 1
                    try:
                        outer_edges.remove({clo_nei1, orig_ver1})
                    except:
                        num_remove_errors += 1

                    try:
                        outer_edges.remove({clo_nei1, orig_ver2})
                    except:
                        num_remove_errors += 1
            else:
                boundary_edges.append(outer_edges[rand_idx])
                outer_edges.remove(outer_edges[rand_idx])


        #rnm case 2
        elif (c1o1 and c1o2) or (c2o1 and c2o2):
            if (case_2_on):
                if (c1o1 and c1o2):
                    new_edge1 = {clo_nei1, orig_ver1}
                    new_edge2 = {clo_nei1, orig_ver2}
                    edges.append(new_edge1)
                    edges.append(new_edge2)

                    center_1 = edge_sampler(vertice_locations[clo_nei1],vertice_locations[orig_ver1])
                    center_2 = edge_sampler(vertice_locations[clo_nei1],vertice_locations[orig_ver2])

                    triangles.append({clo_nei1, orig_ver1, orig_ver2})

                    outer_edges.remove(outer_edges[rand_idx])

                    if point_valid(center_1[0],center_1[1],img):
                        outer_edges.append(new_edge1)
                    else:
                        boundary_edges.append(new_edge1)

                    if point_valid(center_2[0],center_2[1],img):
                        outer_edges.append(new_edge2)
                    else:
                        boundary_edges.append(new_edge2)

                else:
                    new_edge1 = {clo_nei2, orig_ver1}
                    new_edge2 = {clo_nei2, orig_ver2}
                    edges.append(new_edge1)
                    edges.append(new_edge2)

                    center_1 = edge_sampler(vertice_locations[clo_nei2],vertice_locations[orig_ver1])
                    center_2 = edge_sampler(vertice_locations[clo_nei2],vertice_locations[orig_ver2])

                    triangles.append({clo_nei2, orig_ver1, orig_ver2})

                    outer_edges.remove(outer_edges[rand_idx])

                    if point_valid(center_1[0],center_1[1],img):
                        outer_edges.append(new_edge1)
                    else:
                        boundary_edges.append(new_edge1)

                    if point_valid(center_2[0],center_2[1],img):
                        outer_edges.append(new_edge2)
                    else:
                        boundary_edges.append(new_edge2)
            else:
                boundary_edges.append(outer_edges[rand_idx])
                outer_edges.remove(outer_edges[rand_idx])


        #rnm case 3
        else:
            if (case_3_on):
                if {orig_ver1, clo_nei1} not in edges:
                    new_edge = {orig_ver1, clo_nei1}

                    center = edge_sampler(vertice_locations[orig_ver1],vertice_locations[clo_nei1])

                    edges.append(new_edge)

                    triangles.append({orig_ver1, orig_ver2, clo_nei1})

                    outer_edges.remove(outer_edges[rand_idx])

                    try:
                        outer_edges.remove({clo_nei1, orig_ver2})
                    except:
                        num_remove_errors += 1

                    if point_valid(center[0],center[1],img):
                        outer_edges.append(new_edge)
                    else:
                        boundary_edges.append(new_edge)


                elif {orig_ver1, clo_nei2} not in edges:
                    new_edge = {orig_ver1, clo_nei2}

                    center = edge_sampler(vertice_locations[orig_ver1],vertice_locations[clo_nei2])

                    edges.append(new_edge)

                    triangles.append({orig_ver1, orig_ver2, clo_nei2})

                    outer_edges.remove(outer_edges[rand_idx])

                    try:
                        outer_edges.remove({clo_nei2, orig_ver2})
                    except:
                        num_remove_errors += 1

                    if point_valid(center[0],center[1],img):
                        outer_edges.append(new_edge)
                    else:
                        boundary_edges.append(new_edge)

                elif {orig_ver2, clo_nei1} not in edges:
                    new_edge = {orig_ver2, clo_nei1}

                    center = edge_sampler(vertice_locations[orig_ver2],vertice_locations[clo_nei1])

                    edges.append(new_edge)

                    triangles.append({orig_ver1, orig_ver2, clo_nei1})

                    outer_edges.remove(outer_edges[rand_idx])

                    try:
                        outer_edges.remove({clo_nei1, orig_ver1})
                    except:
                        num_remove_errors += 1


                    if point_valid(center[0],center[1],img):
                        outer_edges.append(new_edge)
                    else:
                        boundary_edges.append(new_edge)
                else:
                    new_edge = {orig_ver2, clo_nei2}

                    center = edge_sampler(vertice_locations[orig_ver2],vertice_locations[clo_nei2])

                    edges.append(new_edge)
                    triangles.append({orig_ver1, orig_ver2, clo_nei2})

                    outer_edges.remove(outer_edges[rand_idx])
                    try:
                        outer_edges.remove({clo_nei2, orig_ver1})
                    except:
                        num_remove_errors += 1


                    if point_valid(center[0],center[1],img):
                        outer_edges.append(new_edge)
                    else:
                        boundary_edges.append(new_edge)

            else:
                boundary_edges.append(outer_edges[rand_idx])
                outer_edges.remove(outer_edges[rand_idx])

    return vertice_locations, boundary_edges, edges, outer_edges, triangles, img

In [140]:
# Given a list of edges, this function initializes a Networkx graph then adds edges to the graph
def mesh_to_graph(edges):
    G = nx.Graph()
    for edge in edges:
        e = list(edge)
        G.add_edge(e[0],e[1])

    return G

In [89]:
# source is the first triangle in the triangular mesh
source = create_triangle(5,5)
# records the locations of the first triangle's vertices.
vertice_locations = [(source[0,0],source[0,1]),(source[1,0],source[1,1]),(source[2,0],source[2,1])]
# records the edges of the first triangle's vertices
edges = [{0, 1}, {0, 2}, {1, 2}]
# the first triangle has all three of its edges as outer edges
outer_edges = edges.copy()
# add the first triangle into triangles list which keeps track of all the triangles.
triangles = [{0, 1, 2}]

gs = []
v_locs = []
gs2 = []
# The following for-loop generates a random triangular mesh graph by randomly adding one triangle each time
# Every time a new triangle is added, the new triangular mesh graph will be added to list gs, and updated outer edges will be added to gs2
# Similarly, the position of each node(vertex of triangles) will be added to v_locs
for n in range(100):
    vertice_locations, edges, outer_edges, triangles, operation = create_multi_triangle_rec(vertice_locations, edges, outer_edges, triangles)
    gs.append(mesh_to_graph(edges))
    gs2.append(mesh_to_graph(outer_edges))
    v_locs.append(vertice_locations)


In [90]:
# Draw graph one by one
for i in range(100):
    plt.xlim(-30,30)
    plt.ylim(-30,30)
    nx.draw(gs[i],pos = v_locs[i],node_size = 1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('animation/' + str(i) + '.png')
    plt.clf()

# Draw outer edges one by one
for i in range(100):
    plt.xlim(-30,30)
    plt.ylim(-30,30)
    nx.draw(gs2[i],pos=v_locs[i], node_size=1)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('outer_animation/' + str(i) + '.png')
    plt.clf()


<Figure size 432x288 with 0 Axes>